In [4]:
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.widgets import RangeSlider
import numpy as np

matplotlib.use('TkAgg')


In [5]:
date_url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
df_day = pd.read_csv(date_url)
df_day = df_day[["date_of_interest", "CASE_COUNT", "DEATH_COUNT", "HOSPITALIZED_COUNT"]]
print(df_day.info())

df_day['date_of_interest'] = pd.to_datetime(df_day['date_of_interest'], format='%m/%d/%Y')
print(df_day.info())

print(df_day.head(10))
fig, (ax, ax_slider) = plt.subplots(2, 1, figsize=(14, 8), height_ratios=[6, 1])
# Plot the lines
lines = [ax.plot(df_day['date_of_interest'], df_day['CASE_COUNT'], label='Case Count', color='blue')[0],
         ax.plot(df_day['date_of_interest'], df_day['DEATH_COUNT'], label='Death Count', color='red')[0],
         ax.plot(df_day['date_of_interest'], df_day['HOSPITALIZED_COUNT'], label='Hospitalized Count', color='green')[0]]

# Set title and labels
ax.set_title('New COVID-19 Cases, Deaths, and Hospitalizations Over Time', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Count', fontsize=12)

ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
ax.legend()

# Add grid for better readability
ax.grid(True, linestyle='--', alpha=0.7)

# Create RangeSlider
date_range = (df_day['date_of_interest'].min(), df_day['date_of_interest'].max())
slider = RangeSlider(
    ax_slider, "Date",
    date_range[0].toordinal(), date_range[1].toordinal(),
    valinit=(date_range[0].toordinal(), date_range[1].toordinal())
)

# Update when slider moved
def update(val):
    min_date, max_date = [pd.Timestamp.fromordinal(int(v)) for v in val]
    ax.set_xlim(min_date, max_date)

    # Filter data for the selected date range
    mask = (df_day['date_of_interest'] >= min_date) & (df_day['date_of_interest'] <= max_date)
    df_visible = df_day.loc[mask]

    # Update y-axis limits based on visible data
    y_max = max(df_visible['CASE_COUNT'].max(),
                df_visible['DEATH_COUNT'].max(),
                df_visible['HOSPITALIZED_COUNT'].max())
    ax.set_ylim(0, y_max * 1.1)  # Add 10% padding at the top
    slider.valtext.set_text(str(min_date.date()) + " - " + str(max_date.date()))
    fig.canvas.draw_idle()

slider.on_changed(update)

plt.subplots_adjust(left=0.08, bottom=0.017, right=0.85, top = 0.93, hspace=0.32)

# Adjust layout and display the plot
plt.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1592 entries, 0 to 1591
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date_of_interest    1592 non-null   object
 1   CASE_COUNT          1592 non-null   int64 
 2   DEATH_COUNT         1592 non-null   int64 
 3   HOSPITALIZED_COUNT  1592 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 49.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1592 entries, 0 to 1591
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date_of_interest    1592 non-null   datetime64[ns]
 1   CASE_COUNT          1592 non-null   int64         
 2   DEATH_COUNT         1592 non-null   int64         
 3   HOSPITALIZED_COUNT  1592 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 49.9 KB
None
  date_of_interest  CASE_COUNT  DEATH_COUNT  

In [6]:
fig, (ax, ax_slider) = plt.subplots(2, 1, figsize=(14, 8), height_ratios=[6, 1])
df_day["SUM_CASE"] = np.cumsum(df_day['CASE_COUNT'])
df_day["SUM_DEATH"] = np.cumsum(df_day['DEATH_COUNT'])
df_day["SUM_HOSPITALIZED"] = np.cumsum(df_day['HOSPITALIZED_COUNT'])

print(df_day.head(19))
lines = [ax.plot(df_day['date_of_interest'], df_day["SUM_CASE"], label='Case Sum', color='blue')[0],
        ax.plot(df_day['date_of_interest'], df_day["SUM_DEATH"], label='Death Sum', color='red')[0],
        ax.plot(df_day['date_of_interest'], df_day["SUM_HOSPITALIZED"], label='Hospitalized Sum', color='green')[0]]

ax.set_title('Sum of COVID-19 Cases, Deaths, and Hospitalizations Over Time', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Count', fontsize=12)

ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
ax.legend()

# Add grid for better readability
ax.grid(True, linestyle='--', alpha=0.7)

date_range = (df_day['date_of_interest'].min(), df_day['date_of_interest'].max())
slider = RangeSlider(
    ax_slider, "Date",
    date_range[0].toordinal(), date_range[1].toordinal(),
    valinit=(date_range[0].toordinal(), date_range[1].toordinal())
)

def update(val):
    min_date, max_date = [pd.Timestamp.fromordinal(int(v)) for v in val]
    ax.set_xlim(min_date, max_date)

    # Filter data for the selected date range
    mask = (df_day['date_of_interest'] >= min_date) & (df_day['date_of_interest'] <= max_date)
    df_visible = df_day.loc[mask]

    # Update y-axis limits based on visible data
    y_max = max(df_visible['SUM_CASE'].max(),
                df_visible['SUM_DEATH'].max(),
                df_visible['SUM_HOSPITALIZED'].max())
    y_min = min(df_visible['SUM_CASE'].min(),
                df_visible['SUM_DEATH'].min(),
                df_visible['SUM_HOSPITALIZED'].min())
    ax.set_ylim(y_min*0.8, y_max * 1.1)  # Add 10% padding at the top, 20% padding at the bottom
    slider.valtext.set_text(str(min_date.date()) + " - " + str(max_date.date()))
    fig.canvas.draw_idle()

slider.on_changed(update)
plt.subplots_adjust(left=0.08, bottom=0.017, right=0.85, top = 0.93, hspace=0.32)
# Adjust layout and display the plot
plt.show()

   date_of_interest  CASE_COUNT  DEATH_COUNT  HOSPITALIZED_COUNT  SUM_CASE  \
0        2020-02-29           1            0                   1         1   
1        2020-03-01           0            0                   1         1   
2        2020-03-02           0            0                   2         1   
3        2020-03-03           1            0                   7         2   
4        2020-03-04           5            0                   2         7   
5        2020-03-05           3            0                  14        10   
6        2020-03-06           8            0                   8        18   
7        2020-03-07           7            0                   8        25   
8        2020-03-08          21            0                  18        46   
9        2020-03-09          57            0                  37       103   
10       2020-03-10          69            0                  60       172   
11       2020-03-11         155            1                  78